# 1. Embedding-based Retrieval
基於 embedding 間的相似度計算 (可見 3. Vector Store.ipynb 中關於 embedding 的介紹)，主要有兩種類型，`similarity` 和 `mmr`。

還有一些可設定的參數如 `k` 和 `score_threshold`

In [1]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain_setup import pprint_documents

# fmt: off
docs = [
    Document(page_content="magia", metadata={"source": "魔法少女まどか☆マギカ", "singer": "Kalafina", "genre": "anime"}),
    Document(page_content="烏龍茶是可燃的", metedata={"source": "GRAND BLUE", "author": "井上堅二", "genre": 'anime'}),
    Document(page_content="Knock Knock Knocking on u r heart", metedata={"singer": "李彩演", "genre": 'kpop'}),
    Document(page_content="你指尖躍動的電光，是我此生不變的信仰，唯我超電磁砲永世長存", metadata={"character": "御坂美琴", "genre": "anime"},),
    Document(page_content="Deja vu! I've just been in this place before", metadata={"source": "Initial D", "genre": "anime"}),
    Document(page_content="ふわふわる ふわふわり", metadata={"name": "戀愛循環", "genre": "anime"})
]
# fmt: on

vectorstore = Qdrant.from_documents(
    docs,
    embedding=OpenAIEmbeddings(),
    location=":memory:",
)

## 1.1 Similarity (相似度)
- 預設演算法

1. 比較 embedding 算出相似度

2. 只挑出相似度大於等於 `score_threshold` 的文件 (documents) (如果 `score_threshold` 不為 `None` 的話)

3. 再挑出其中相似度前 k 高的文件

In [2]:
retriver = vectorstore.as_retriever(
    search_type="similarity",  # default
    search_kwargs={
        "filter": {'genre': 'anime'},  # Filter by metadata
        "score_threshold": None,
        "k": 2,
        # More kwargs for `Qdrant.similarity_search` or `QdrantClient.search` like `offset`, `consistency`, ...
    },
)
pprint_documents(retriver.get_relevant_documents("神曲"))

Document 1:

ふわふわる ふわふわり

Metadata:{'genre': 'anime', 'name': '戀愛循環'}
----------------------------------------------------------------------------------------------------
Document 2:

你指尖躍動的電光，是我此生不變的信仰，唯我超電磁砲永世長存

Metadata:{'character': '御坂美琴', 'genre': 'anime'}


更複雜的針對詮釋資料 (metadata) 的過濾 (filtering)，可以自己做 `qdrant_client.http.models.filter` 傳進去。可參考 [Qdrant官方文件](https://qdrant.tech/documentation/concepts/filtering/)

## 1.2 Maximum marginal relevance (最大邊界相關算法)
1. 比較 embedding 算出相似度

2. 只挑出相似度大於等於 `score_threshold` 的文件 (documents) (如果 `score_threshold` 不為 `None` 的話)

3. 再挑出其中相似度前 `fetch_k` 高的文件

4. 再對剩下的文件做以 MMR 演算法取 `k` 個
   - 簡言之是考慮到候選文件間的相似度，避免相似度高的文件都是差不多的文件。
   
   - $ MMR(q,D,R) = \argmax_{d_i \in D} \left[ \lambda sim(q,d_i) - (1-\lambda)\max_{d_j \in R}sim(d_i, d_j) \right]  $
      - q: query
      - D: 還沒被抽取的文件的集合
      - R: 已抽取的文件的集合
      - lambda ($\lambda$): 
         - 可調整的參數，介於 0~1 之間。
         - 是一個 tradeoff。越高則總體跟問題 (query) 越像，但文件間多樣性越低，越低則跟總體跟問題越不像，但文件間多樣性越高
      - 會一個一個迭代地 (iteratively) 的選擇

In [3]:
retriver = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "filter": {'genre': 'anime'}, 
        "score_threshold": None,
        "fetch_k": 50,
        "lambda_mult": 0.5,  # 多樣性，越低越多樣 (diverse)
        "k": 3,
        # More kwargs for `Qdrant.similarity_search` or `QdrantClient.search` like `consistency`, ...
    },
)
pprint_documents(retriver.get_relevant_documents("神曲"))

AttributeError: 'dict' object has no attribute 'must'

## 1.3 Similarity Score Threshold (相似度分數閥值)

想要抽取 (retreive) 不固定的數量的文件 (documents)，其實就是把 `k` 設非常大，其就只會靠 `score_threshold` 來挑選了

In [4]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # default
    search_kwargs={
        "score_threshold": 0.72,  
        "k": 9999,
        # More kwargs for `Qdrant.similarity_search` or `QdrantClient.search` like `offset`, `consistency`, ...
    },
)
pprint_documents(retriever.get_relevant_documents("神曲"))

Document 1:

ふわふわる ふわふわり

Metadata:{'genre': 'anime', 'name': '戀愛循環'}
----------------------------------------------------------------------------------------------------
Document 2:

你指尖躍動的電光，是我此生不變的信仰，唯我超電磁砲永世長存

Metadata:{'character': '御坂美琴', 'genre': 'anime'}
----------------------------------------------------------------------------------------------------
Document 3:

magia

Metadata:{'genre': 'anime', 'singer': 'Kalafina', 'source': '魔法少女まどか☆マギカ'}
----------------------------------------------------------------------------------------------------
Document 4:

Knock Knock Knocking on u r heart

Metadata:{}


In [5]:
result = vectorstore.similarity_search_with_relevance_scores('神曲', k=100)
for document, score in result:
    print(document.page_content, score)

ふわふわる ふわふわり 0.7890049945006924
你指尖躍動的電光，是我此生不變的信仰，唯我超電磁砲永世長存 0.7731011271614837
magia 0.7585180099997237
Knock Knock Knocking on u r heart 0.7285625185885174
烏龍茶是可燃的 0.7183404786994874
Deja vu! I've just been in this place before 0.7022273482632053


# 其他不同的 Retriever
雖然藉由比較向量相似度算出相關性分數的 Embedding-based Retrieval 是主流，但其實還有很多不同的算法。而之後也會介紹到我們並不是只能挑一種算法...
|           | 判斷相似的依據      | 速度 | 例如                      |
|-----------|---------------------|------|---------------------------|
| lexical   | 表面文面            | 快   | 關鍵字搜尋、BM25          |
| semantics | 深層語意            | 中   | Embedding-based Retrieval |
| cross     | 表面文面 + 深層語意 | 慢   | cross-encoder             |

In [6]:
# 單靠詞彙的統計來抽取 (retrieve) 的演算法
from langchain.retrievers import BM25Retriever, TFIDFRetriever

# 其他依靠向量的抽取 (retrieve) 演算法
from langchain.retrievers import KNNRetriever, SVMRetriever

# 針對某些現成服務製作的現成 retriever
from langchain.retrievers import ArxivRetriever, GoogleCloudEnterpriseSearchRetriever